# Testing Comments and Duplication

In [ ]:
import os

from felt_python import (
    create_map,
    delete_map,
    export_comments,
    resolve_comment,
    delete_comment,
    duplicate_layers,
    upload_file,
    list_layers
)

os.environ["FELT_API_TOKEN"] = "<YOUR_API_TOKEN>"

# Create maps for testing

In [ ]:
# Create source map
source_map = create_map(
    title="Source Map for Testing",
    lat=40,
    lon=-3,
    zoom=5,
    public_access="view_and_comment"  # Allow comments
)
source_map_id = source_map["id"]

# Create destination map (for duplication)
dest_map = create_map(
    title="Destination Map for Testing",
    lat=40,
    lon=-3,
    zoom=5,
    public_access="private"
)
dest_map_id = dest_map["id"]

print(f"Source map URL: {source_map['url']}")
print(f"Destination map URL: {dest_map['url']}")

# Upload a layer to the source map

In [ ]:
# Upload a layer to the source map
layer_resp = upload_file(
    map_id=source_map_id,
    file_name="tests/fixtures/null-island-points.geojson",
    layer_name="Layer to duplicate"
)
layer_id = layer_resp["layer_id"]

# Wait for layer processing
import time
print("Waiting for layer to process...")
time.sleep(10)

# Export comments

Note: This example assumes there are comments on the map. If you've just created the map, 
you'll need to add some comments through the Felt UI before running this code.

In [ ]:
# Export comments as JSON (default)
comments_json = export_comments(source_map_id)
print(f"Number of comment threads: {len(comments_json)}")

# Export comments as CSV
# comments_csv = export_comments(source_map_id, format="csv")

# Resolve a comment

Note: Replace with a real comment ID from your map

In [ ]:
# If you have comments from the export above, you can use one of those IDs
if len(comments_json) > 0:
    comment_id = comments_json[0].get("id")
    if comment_id:
        resolve_result = resolve_comment(source_map_id, comment_id)
        print(f"Comment resolved: {resolve_result}")
else:
    print("No comments available to resolve. Add comments through the Felt UI.")

# Delete a comment

Note: Replace with a real comment ID from your map

In [ ]:
# If you have comments from the export above, you can use one of those IDs
if len(comments_json) > 1:
    comment_id = comments_json[1].get("id")
    if comment_id:
        delete_comment(source_map_id, comment_id)
        print(f"Comment deleted: {comment_id}")
else:
    print("Not enough comments available to delete. Add comments through the Felt UI.")

# Export comments again to verify changes

In [ ]:
updated_comments = export_comments(source_map_id)
print(f"Number of comment threads after modifications: {len(updated_comments)}")

# Duplicate a layer from source map to destination map

In [ ]:
# Check the layers on the source map
source_layers = list_layers(source_map_id)
print(f"Source map has {len(source_layers)} layers")

if len(source_layers) > 0:
    source_layer_id = source_layers[0]["id"]
    
    # Duplicate the layer
    duplication_params = [
        {
            "source_layer_id": source_layer_id,
            "destination_map_id": dest_map_id
        }
    ]
    
    duplication_result = duplicate_layers(duplication_params)
    print(f"Duplicated {len(duplication_result['layers'])} layers")
    print(f"Duplicated {len(duplication_result['layer_groups'])} layer groups")
    
    # Check if duplication worked
    dest_layers = list_layers(dest_map_id)
    print(f"Destination map now has {len(dest_layers)} layers")

# Duplicate a layer group

Note: This assumes you have a layer group on your source map

In [ ]:
# If the source map has layer groups, you can duplicate a group
from felt_python import list_layer_groups

source_groups = list_layer_groups(source_map_id)
print(f"Source map has {len(source_groups)} layer groups")

if len(source_groups) > 0:
    source_group_id = source_groups[0]["id"]
    
    # Duplicate the layer group
    group_duplication_params = [
        {
            "source_layer_group_id": source_group_id,
            "destination_map_id": dest_map_id
        }
    ]
    
    group_duplication_result = duplicate_layers(group_duplication_params)
    print(f"Duplicated {len(group_duplication_result['layers'])} layers")
    print(f"Duplicated {len(group_duplication_result['layer_groups'])} layer groups")

# Clean up

In [ ]:
# Delete both maps
delete_map(source_map_id)
delete_map(dest_map_id)